Transcript1 - [link](https://www.youtube.com/watch?v=BDIRabVP24o) (ENGLISH SPEECH | ELON MUSK: Think Big & Dream Even Bigger)

Transcript2 - [link](https://www.englishspeecheschannel.com/english-speeches/elon-musk-speech/) (Elon Musk Speech: Future, A.I., and Mars)

Transcript3 - [link](https://www.youtube.com/watch?v=IgKWPdJWuBQ&t=21s) (Элон Маск: Элон Маск: Человек, создавший Tesla, SpaceX, SolarCity...)

Transcript4 - [link](https://www.youtube.com/watch?v=zIwLWfaAg-8) (Elon Musk: The future we're building -- and boring | TED)

Transcript5 - [link](https://www.youtube.com/watch?v=H7Uyfqi_TE8) (Making Humans a Multiplanetary Species)

In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
import os

In [2]:
def set_sentences(text):
    new = ''
    for i in range(len(text) - 1):
        if text[i] == '.' and text[i + 1] == ' ':
            new += '*'
        else:
            new += text[i]
    return new

In [3]:
def transform1():
    f = open('Data/Data1.txt', 'r')
    
    text = f.read()
    sentences = []
    for sentence in text.split('.\n'):
        sentences.append(sentence.strip().replace('\n', ' '))
        
    f.close()
    
    return sentences

In [118]:
def get_podcast1(web):
    
    link = 'https://www.rev.com/blog/transcripts/joe-rogan-elon-musk-podcast-transcript-may-7-2020'
    web.get(link)
    
    sleep(4)
    
    sentences = []
    main = web.find_element_by_class_name('fl-callout-text')
    for phrase in web.find_element_by_class_name('fl-callout-text').find_elements_by_css_selector('p'):
        if len(phrase.text.split('\n')) == 2:
            name, text = phrase.text.split('\n')
        if 'Elon Musk:' in name:
            for sent in text.split('.'):
                sentences.append(sent)
    return sentences




def get_podcast2(web):
    link = 'https://sonix.ai/resources/full-transcript-joe-rogan-experience-elon-musk/'
    web.get(link)
    
    sleep(5)
    
    sentences = []
    for x in web.find_elements_by_class_name('sonix--transcript-exchange'):
        if len(x.text.split('\n')) == 3:
            _, name, phrase = x.text.split('\n')
            if name == 'Elon Musk':
                for sent in phrase.split('.'):
                    sentences.append(sent)

    return sentences


def parse():
    script_dir = os.path.dirname(os.path.realpath('chromedriver'))
    webdriver_path = os.path.join(script_dir, 'chromedriver')
    web = webdriver.Chrome(webdriver_path)
    
    link = 'https://www.englishspeecheschannel.com/english-speeches/elon-musk-speech/'
    web.get(link)
    sleep(5)
    
    read_more = web.find_element_by_class_name('read-link')
    read_more.click()
    
    part1 = []
    header = web.find_element_by_css_selector('#qtcontents > div > div.col.s12.m12.l8 > div > div:nth-child(3) > div > div > div > div > div')
    for block in header.find_elements_by_css_selector('p')[3:6]: ## part1
        part1.append(block.text)

    part2 = web.find_element_by_class_name('read_div').text.split('\n') ## part2
    text = part1 + part2

    sentences = []
    for block in text:
        if (block[-1] == '?' and len(block.split('.')) < 10) or (len(block.split('.')) < 6):
            continue

        block = set_sentences(block)
            
        for sentence in block.split('*'):
            sentences.append(sentence)

    sentences = list(filter(lambda x: ' – ' not in x, sentences))
    
    sentences1 = get_podcast1(web)
    sentences2 = get_podcast2(web)
    
    return sentences + sentences1 + sentences2



In [119]:
def transform2(filename):
    f = open(filename, 'r')
    text = f.read().replace('\n\n', ' ').replace('(Laughter)', '').replace('(Applause)', '')
    f.close()

    first = text.split('EM:')[0]
    other = text.split('EM:')[1:]

    blocks = []
    blocks.append(first.split('Elon Musk:')[1].split('CA:')[0])

    for block in other:
        blocks.append(block.split('CA:')[0].replace('\n', ''))


    sentences = []
    for block in blocks:
        
        block = set_sentences(block)
        for sent in block.split('*'):
            sentences.append(sent)
            
    return sentences

In [120]:
def transform3():
    f = open('Data/Data4.txt', 'r')

    text = f.read().replace('\n\n', ' ').replace('(Laughter)', '').replace('[APPLAUSE]', '')
    text = text.split('ELON MUSK:')[1]
    text = set_sentences(text)
    sentences = text.split('*')

    f.close()
    
    return sentences

In [ ]:
def find_frequent_words(speeches):
    count_unique = {}
    for sent in speeches:
        for word in sent.split():
            if word in count_unique:
                count_unique[word] += 1
            else:
                count_unique[word] = 1


    sorted_dict = {}
    sorted_keys = sorted(count_unique, key=count_unique.get, reverse=True)

    for w in sorted_keys:
        sorted_dict[w] = count_unique[w]

    stop_words = list(sorted_dict.keys())[:80]
    return stop_words


def clear_sent(sent, stop_words):
    split_str = sent.split()
    filtered_str = list(filter(lambda s: s not in stop_words, split_str))

    return filtered_str

In [149]:
speeches = []
speeches += transform1()
speeches += transform2('Data/Data2.txt')
speeches += transform2('Data/Data3.txt')
speeches += transform3()
speeches += parse()

In [184]:
stop_words = find_frequent_words(speeches)
speeches_cleared = list(filter(lambda x: len(clear_sent(x, stop_words)) > 6 and '--' not in x ,speeches))

In [185]:
df = pd.DataFrame({'Speech': speeches_cleared})
df['Speech'] = df['Speech'].apply(lambda x: x.replace('\\', ''))
df.to_csv('Elon_phrases.csv')

In [186]:
df.head()

,Speech
0,"So, I thought — I was trying to think what’s t..."
1,"And I thought, perhaps tell the story of how I..."
2,"I often find myself wondering, how did this ha..."
3,And the reason I thought that was because I re...
4,"If you go back say, 300 years, the things we t..."
